Осуществим предобработку данных с Твиттера, чтобы отчищенный данные в дальнейшем использовать для задачи классификации. Данный датасет содержит негативные (label = 1) и нейтральные (label = 0) высказывания.
Для работы объединим train_df и test_df.

Задания:

1) Заменим html-сущности (к примеру: &lt; &gt; &amp;). "&lt;" заменим на “<” и "&amp;" заменим на “&”)""". Сделаем это с помощью HTMLParser.unescape(). Всю предобработку делаем в новом столбце 'clean_tweet'

2) Удалим @user из всех твитов с помощью паттерна "@[\w]*". Для этого создадим функцию: 
 - для того, чтобы найти все вхождения паттерна в тексте, необходимо использовать re.findall(pattern, input_txt)
 - для для замены @user на пробел, необходимо использовать re.sub()
при применении функции необходимо использовать np.vectorize(function).

3) Изменим регистр твитов на нижний с помощью .lower().

4) Заменим сокращения с апострофами (пример: ain't, can't) на пробел, используя apostrophe_dict. Для этого необходимо сделать функцию: для каждого слова в тексте проверить (for word in text.split()), если слово есть в словаре apostrophe_dict в качестве ключа (сокращенного слова), то заменить ключ на значение (полную версию слова).

5) Заменим сокращения на их полные формы, используя short_word_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

6) Заменим эмотиконы (пример: ":)" = "happy") на пробелы, используя emoticon_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

7) Заменим пунктуацию на пробелы, используя re.sub() и паттерн r'[^\w\s]'.

8) Заменим спец. символы на пробелы, используя re.sub() и паттерн r'[^a-zA-Z0-9]'.

9) Заменим числа на пробелы, используя re.sub() и паттерн r'[^a-zA-Z]'.

10) Удалим из текста слова длиной в 1 символ, используя ' '.join([w for w in x.split() if len(w)>1]).

11) Поделим твиты на токены с помощью nltk.tokenize.word_tokenize, создав новый столбец 'tweet_token'.

12) Удалим стоп-слова из токенов, используя nltk.corpus.stopwords. Создадим столбец 'tweet_token_filtered' без стоп-слов.

13) Применим стемминг к токенам с помощью nltk.stem.PorterStemmer. Создадим столбец 'tweet_stemmed' после применения стемминга.

14) Применим лемматизацию к токенам с помощью nltk.stem.wordnet.WordNetLemmatizer. Создадим столбец 'tweet_lemmatized' после применения лемматизации.

15) Сохраним результат предобработки в pickle-файл.

In [1]:
apostrophe_dict = {
"ain't": "am not / are not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is",
"i'd": "I had / I would",
"i'd've": "I would have",
"i'll": "I shall / I will",
"i'll've": "I shall have / I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}



short_word_dict = {
"121": "one to one",
"a/s/l": "age, sex, location",
"adn": "any day now",
"afaik": "as far as I know",
"afk": "away from keyboard",
"aight": "alright",
"alol": "actually laughing out loud",
"b4": "before",
"b4n": "bye for now",
"bak": "back at the keyboard",
"bf": "boyfriend",
"bff": "best friends forever",
"bfn": "bye for now",
"bg": "big grin",
"bta": "but then again",
"btw": "by the way",
"cid": "crying in disgrace",
"cnp": "continued in my next post",
"cp": "chat post",
"cu": "see you",
"cul": "see you later",
"cul8r": "see you later",
"cya": "bye",
"cyo": "see you online",
"dbau": "doing business as usual",
"fud": "fear, uncertainty, and doubt",
"fwiw": "for what it's worth",
"fyi": "for your information",
"g": "grin",
"g2g": "got to go",
"ga": "go ahead",
"gal": "get a life",
"gf": "girlfriend",
"gfn": "gone for now",
"gmbo": "giggling my butt off",
"gmta": "great minds think alike",
"h8": "hate",
"hagn": "have a good night",
"hdop": "help delete online predators",
"hhis": "hanging head in shame",
"iac": "in any case",
"ianal": "I am not a lawyer",
"ic": "I see",
"idk": "I don't know",
"imao": "in my arrogant opinion",
"imnsho": "in my not so humble opinion",
"imo": "in my opinion",
"iow": "in other words",
"ipn": "I’m posting naked",
"irl": "in real life",
"jk": "just kidding",
"l8r": "later",
"ld": "later, dude",
"ldr": "long distance relationship",
"llta": "lots and lots of thunderous applause",
"lmao": "laugh my ass off",
"lmirl": "let's meet in real life",
"lol": "laugh out loud",
"ltr": "longterm relationship",
"lulab": "love you like a brother",
"lulas": "love you like a sister",
"luv": "love",
"m/f": "male or female",
"m8": "mate",
"milf": "mother I would like to fuck",
"oll": "online love",
"omg": "oh my god",
"otoh": "on the other hand",
"pir": "parent in room",
"ppl": "people",
"r": "are",
"rofl": "roll on the floor laughing",
"rpg": "role playing games",
"ru": "are you",
"shid": "slaps head in disgust",
"somy": "sick of me yet",
"sot": "short of time",
"thanx": "thanks",
"thx": "thanks",
"ttyl": "talk to you later",
"u": "you",
"ur": "you are",
"uw": "you’re welcome",
"wb": "welcome back",
"wfm": "works for me",
"wibni": "wouldn't it be nice if",
"wtf": "what the fuck",
"wtg": "way to go",
"wtgp": "want to go private",
"ym": "young man",
"gr8": "great"
}


emoticon_dict = {
":)": "happy",
":‑)": "happy",
":-]": "happy",
":-3": "happy",
":->": "happy",
"8-)": "happy",
":-}": "happy",
":o)": "happy",
":c)": "happy",
":^)": "happy",
"=]": "happy",
"=)": "happy",
"<3": "happy",
":-(": "sad",
":(": "sad",
":c": "sad",
":<": "sad",
":[": "sad",
">:[": "sad",
":{": "sad",
">:(": "sad",
":-c": "sad",
":-< ": "sad",
":-[": "sad",
":-||": "sad"
}

In [2]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from nltk import word_tokenize
from nltk.corpus import stopwords
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')

from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet

import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
import os

import html.parser    

pd.set_option('display.max_colwidth', -1)

In [3]:
train_df = pd.read_csv('train_tweets.csv')
train_df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run
1,2,0,@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦
4,5,0,factsguide: society now #motivation


In [4]:
test_df = pd.read_csv('test_tweets.csv')
test_df.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedication #willpower to find #newmaterialsâ¦
1,31964,@user #white #supremacists want everyone to see the new â #birdsâ #movie â and hereâs why
2,31965,safe ways to heal your #acne!! #altwaystoheal #healthy #healing!!
3,31966,"is the hp and the cursed child book up for reservations already? if yes, where? if no, when? ððð #harrypotter #pottermore #favorite"
4,31967,"3rd #bihday to my amazing, hilarious #nephew eli ahmir! uncle dave loves you and missesâ¦"


In [5]:
combine_df = train_df.append(test_df, ignore_index = True, sort = False)
combine_df.head()

,id,label,tweet
0,1,0.0,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run
1,2,0.0,@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked
2,3,0.0,bihday your majesty
3,4,0.0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦
4,5,0.0,factsguide: society now #motivation


In [6]:
print(combine_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49159 entries, 0 to 49158
Data columns (total 3 columns):
id       49159 non-null int64
label    31962 non-null float64
tweet    49159 non-null object
dtypes: float64(1), int64(1), object(1)
memory usage: 1.1+ MB
None


## ДЗ

In [7]:
class preprocessing:

    def __init__(self, **param):
        if 'file_name' in param:
            self.file_name = param['file_name']
            self.df_load = pd.read_csv(self.file_name)
            self.df = ''
        else:
            return
        
    
    def _replace(self, text, dictonary, patern):
        results = re.findall(patern, text)
        for item in results:
            word = dictonary.get(item)
            if word != None:
                text = re.sub(item, word, text)
        return text

    def get_patern_emoji(self):
        simbol = ['[', '\\', '^', '$', '.', '|', '?', '*', '+', '(', ')']

        def __replace(s):
            new = ''
            for item in s:
                if item in simbol:
                    new += f'\{item}'
                else:
                    new += item
            return new

        return '|'.join([f'({__replace(x)})' for x in emoticon_dict])
    
    def preproces(self):

        df = self.df_load.copy()

        df['tweet'] = df['tweet'].astype('str') 
 
        # 1. Заменим html-сущности (к примеру: &lt; &gt; &amp;). "&lt;" заменим на “<” и "&amp;" заменим на “&”)""". Сделаем это с помощью HTMLParser.unescape()

        df['clean_tweet'] = html.parser.HTMLParser().unescape(df['tweet'])


        # 2. Удалим @user из всех твитов с помощью паттерна "@[\w]*". Для этого создадим функцию: 
        #  - для того, чтобы найти все вхождения паттерна в тексте, необходимо использовать re.findall(pattern, input_txt)
        #  - для для замены @user на пробел, необходимо использовать re.sub()
        # при применении функции необходимо использовать np.vectorize(function)


        df['clean_tweet'] = df['clean_tweet'].apply(lambda x: re.sub('@[\w]*','', x))

        # 3. Изменим регистр твитов на нижний с помощью .lower()

        df['clean_tweet'] = df['clean_tweet'].str.lower()

        # 4. Заменим сокращения с апострофами (пример: ain't, can't) на пробел, используя apostrophe_dict. 
        # Для этого необходимо сделать функцию: для каждого слова в тексте проверить (for word in text.split()), 
        # если слово есть в словаре apostrophe_dict в качестве ключа (сокращенного слова), 
        # то заменить ключ на значение (полную версию слова).

        df['clean_tweet'] = df['clean_tweet'].apply(lambda x: self._replace(x, apostrophe_dict, "[\w]*'[\w]*"))

        # 5. Заменим сокращения на их полные формы, используя short_word_dict. 
        # Для этого воспользуемся функцией, используемой в предыдущем пункте.

        df['clean_tweet'] = df['clean_tweet'].apply(lambda x: self._replace(x, short_word_dict, "[ ]"))

        # 6. Заменим эмотиконы (пример: ":)" = "happy") на пробелы, используя emoticon_dict. 
        # Для этого воспользуемся функцией, используемой в предыдущем пункте.
        patern = self.get_patern_emoji()
        df['clean_tweet'] = df['clean_tweet'].apply(lambda x: self._replace(x, emoticon_dict,patern))
        #'|'.join(['('+x+')' for x in emoticon_dict])

        # 7. Заменим пунктуацию на пробелы, используя re.sub() и паттерн r'[^\w\s]'

        df['clean_tweet'] = df['clean_tweet'].apply(lambda x: re.sub(r'[^\w\s]',' ', x))

        # 8. Заменим спец. символы на пробелы, используя re.sub() и паттерн r'[^a-zA-Z0-9]'

        df['clean_tweet'] = df['clean_tweet'].apply(lambda x: re.sub(r'[^a-zA-Z0-9]',' ', x))

        # 9. Заменим числа на пробелы, используя re.sub() и паттерн r'[^a-zA-Z]'

        df['clean_tweet'] = df['clean_tweet'].apply(lambda x: re.sub(r'[^a-zA-Z]',' ', x))

        # 10. Удалим из текста слова длиной в 1 символ, используя ' '.join([w for w in x.split() if len(w)>1])

        df['clean_tweet'] = df['clean_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>1]))

        # 11. Поделим твиты на токены с помощью nltk.tokenize.word_tokenize, создав новый столбец 'tweet_token'.

        df['tweet_token'] = df['clean_tweet'].apply(lambda x: str(word_tokenize(x)))
        df['tweet_token'] = df['tweet_token'].astype('str') 

        # 12. Удалим стоп-слова из токенов, используя nltk.corpus.stopwords. 
        # Создадим столбец 'tweet_token_filtered' без стоп-слов.
        sw = set(stopwords.words("english"))
        df['tweet_token_filtered'] = df['tweet_token'].apply(lambda x: str(list(set(eval(x)).difference(sw))))
        # 

        # 13. Применим стемминг к токенам с помощью nltk.stem.PorterStemmer. 
        # Создадим столбец 'tweet_stemmed' после применения стемминга.

        porter = PorterStemmer()
        df['tweet_stemmed'] = df['tweet_token_filtered'].apply(lambda x: str([porter.stem(i) for i in eval(x)]))

        # 14. Применим лемматизацию к токенам с помощью nltk.stem.wordnet.WordNetLemmatizer. 
        # Создадим столбец 'tweet_lemmatized' после применения лемматизации.

        lemmatizer = WordNetLemmatizer()
        df['tweet_lemmatized'] = df['tweet_token_filtered'
                                   ].apply(lambda x: str([lemmatizer.lemmatize(i,wordnet.VERB
                                                                              ) for i in eval(x)]
                                                        )
                                          )
        self.df = df
    # 15. Сохраним результат предобработки в pickle-файл.        
    def _save(self, path = ''):
        self.df.to_pickle(f'{path}{self.file_name}.pkl')

Загружаем тренировочный файл

In [8]:
pr = preprocessing(file_name = 'train_tweets.csv')
pr.df_load.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run
1,2,0,@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦
4,5,0,factsguide: society now #motivation


In [9]:
pr.preproces()
pr._save()
pr.df.head()

,id,label,tweet,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when father is dysfunctional and is so selfish he drags his kids into his dysfunction run,"['when', 'father', 'is', 'dysfunctional', 'and', 'is', 'so', 'selfish', 'he', 'drags', 'his', 'kids', 'into', 'his', 'dysfunction', 'run']","['run', 'father', 'selfish', 'kids', 'dysfunctional', 'dysfunction', 'drags']","['run', 'father', 'selfish', 'kid', 'dysfunct', 'dysfunct', 'drag']","['run', 'father', 'selfish', 'kid', 'dysfunctional', 'dysfunction', 'drag']"
1,2,0,@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked,thanks for lyft credit cannot use cause they do not offer wheelchair vans in pdx disapointed getthanked,"['thanks', 'for', 'lyft', 'credit', 'can', 'not', 'use', 'cause', 'they', 'do', 'not', 'offer', 'wheelchair', 'vans', 'in', 'pdx', 'disapointed', 'getthanked']","['lyft', 'offer', 'wheelchair', 'vans', 'cause', 'thanks', 'getthanked', 'pdx', 'use', 'disapointed', 'credit']","['lyft', 'offer', 'wheelchair', 'van', 'caus', 'thank', 'getthank', 'pdx', 'use', 'disapoint', 'credit']","['lyft', 'offer', 'wheelchair', 'vans', 'cause', 'thank', 'getthanked', 'pdx', 'use', 'disapointed', 'credit']"
2,3,0,bihday your majesty,bihday your majesty,"['bihday', 'your', 'majesty']","['majesty', 'bihday']","['majesti', 'bihday']","['majesty', 'bihday']"
3,4,0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦,model love take with all the time in ur,"['model', 'love', 'take', 'with', 'all', 'the', 'time', 'in', 'ur']","['model', 'ur', 'take', 'time', 'love']","['model', 'ur', 'take', 'time', 'love']","['model', 'ur', 'take', 'time', 'love']"
4,5,0,factsguide: society now #motivation,factsguide society now motivation,"['factsguide', 'society', 'now', 'motivation']","['factsguide', 'motivation', 'society']","['factsguid', 'motiv', 'societi']","['factsguide', 'motivation', 'society']"


Загружаем тестовый файл

In [10]:
pr = preprocessing(file_name = 'test_tweets.csv')
pr.preproces()
pr._save()
pr.df.head()

,id,tweet,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,31963,#studiolife #aislife #requires #passion #dedication #willpower to find #newmaterialsâ¦,studiolife aislife requires passion dedication willpower to find newmaterials,"['studiolife', 'aislife', 'requires', 'passion', 'dedication', 'willpower', 'to', 'find', 'newmaterials']","['requires', 'aislife', 'passion', 'newmaterials', 'find', 'willpower', 'studiolife', 'dedication']","['requir', 'aislif', 'passion', 'newmateri', 'find', 'willpow', 'studiolif', 'dedic']","['require', 'aislife', 'passion', 'newmaterials', 'find', 'willpower', 'studiolife', 'dedication']"
1,31964,@user #white #supremacists want everyone to see the new â #birdsâ #movie â and hereâs why,white supremacists want everyone to see the new birds movie and here why,"['white', 'supremacists', 'want', 'everyone', 'to', 'see', 'the', 'new', 'birds', 'movie', 'and', 'here', 'why']","['birds', 'everyone', 'white', 'see', 'want', 'movie', 'new', 'supremacists']","['bird', 'everyon', 'white', 'see', 'want', 'movi', 'new', 'supremacist']","['bird', 'everyone', 'white', 'see', 'want', 'movie', 'new', 'supremacists']"
2,31965,safe ways to heal your #acne!! #altwaystoheal #healthy #healing!!,safe ways to heal your acne altwaystoheal healthy healing,"['safe', 'ways', 'to', 'heal', 'your', 'acne', 'altwaystoheal', 'healthy', 'healing']","['ways', 'healing', 'safe', 'healthy', 'altwaystoheal', 'heal', 'acne']","['way', 'heal', 'safe', 'healthi', 'altwaystoh', 'heal', 'acn']","['ways', 'heal', 'safe', 'healthy', 'altwaystoheal', 'heal', 'acne']"
3,31966,"is the hp and the cursed child book up for reservations already? if yes, where? if no, when? ððð #harrypotter #pottermore #favorite",is the hp and the cursed child book up for reservations already if yes where if no when harrypotter pottermore favorite,"['is', 'the', 'hp', 'and', 'the', 'cursed', 'child', 'book', 'up', 'for', 'reservations', 'already', 'if', 'yes', 'where', 'if', 'no', 'when', 'harrypotter', 'pottermore', 'favorite']","['hp', 'already', 'book', 'cursed', 'child', 'pottermore', 'reservations', 'favorite', 'harrypotter', 'yes']","['hp', 'alreadi', 'book', 'curs', 'child', 'pottermor', 'reserv', 'favorit', 'harrypott', 'ye']","['hp', 'already', 'book', 'curse', 'child', 'pottermore', 'reservations', 'favorite', 'harrypotter', 'yes']"
4,31967,"3rd #bihday to my amazing, hilarious #nephew eli ahmir! uncle dave loves you and missesâ¦",rd bihday to my amazing hilarious nephew eli ahmir uncle dave loves you and misses,"['rd', 'bihday', 'to', 'my', 'amazing', 'hilarious', 'nephew', 'eli', 'ahmir', 'uncle', 'dave', 'loves', 'you', 'and', 'misses']","['hilarious', 'uncle', 'ahmir', 'nephew', 'dave', 'rd', 'misses', 'amazing', 'loves', 'bihday', 'eli']","['hilari', 'uncl', 'ahmir', 'nephew', 'dave', 'rd', 'miss', 'amaz', 'love', 'bihday', 'eli']","['hilarious', 'uncle', 'ahmir', 'nephew', 'dave', 'rd', 'miss', 'amaze', 'love', 'bihday', 'eli']"


## ДЗ 3.1

In [79]:
# !pip install en_core_web_md
# !python -m spacy download en_core_web_sm
#from spacy import displacy#библиолтека для визуализации текста и NER
import spacy
nlp = spacy.load("en_core_web_sm")

# displacy.render(doc, jupyter=True, style='ent')

df = pd.DataFrame()

def get_label(text):
    global df
    doc  = nlp(text)
    for entity in doc.ents:
        df = df.append({'Type':entity.label_, 'Value':entity.text}, ignore_index=True)
    return doc.ents[0].label_ if len(doc.ents)>0 else None
    

In [ ]:
pr.df['tweet_ner'] = pr.df['clean_tweet'].apply(lambda x: get_label(x))

In [55]:
pr.df.groupby(['tweet_ner']).size().reset_index(name='counts'
                                               ).sort_values(by=['counts'], 
                                                             ascending=False)

,tweet_ner,counts
1,DATE,2760
13,PERSON,2006
11,ORG,1509
4,GPE,888
16,TIME,486
9,NORP,326
0,CARDINAL,274
10,ORDINAL,167
3,FAC,61
7,LOC,40


In [110]:
df_top = df.loc[(df['Type']=='ORG')|(df['Type']=='PERSON')]

In [143]:
df_top = df_top.groupby(['Value','Type']).size().reset_index(name='counts').sort_values(by=['counts'], 
                                                             ascending=False).head(20)
df_top

,Value,Type,counts
1384,hea,PERSON,35
522,bong bing,ORG,32
1472,hu,PERSON,26
212,app,ORG,24
1259,gop,ORG,24
1086,feminismiscancer feminismisterrorism feminismmuktbharat,PERSON,20
1559,islam,ORG,18
387,bihday,PERSON,17
3037,sta,ORG,16
133,amazon,ORG,14


## 3.2

In [199]:
df_nltk = pd.DataFrame()

def get_label_nltk(text):
    global df_nltk
    text = f'{text[0].upper()}{text[1:]}'
    for sent in nltk.sent_tokenize(text):
        for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
            if hasattr(chunk, 'label'):
                df_nltk = df_nltk.append({'Type':chunk.label(), 'Value':chunk[0][0]}, ignore_index=True)
                return chunk.label()
    return None

In [201]:
pr.df['tweet_ner_nltk'] = pr.df['clean_tweet'].apply(lambda x: get_label_nltk(x))

,Type,Value
0,GPE,Studiolife
1,GPE,White
2,GPE,Safe
3,GPE,Rd
4,GPE,Finished
...,...,...
6520,GPE,Good
6521,GPE,Black
6522,GPE,Damn
6523,GPE,Thought


In [202]:
pr.df.groupby(['tweet_ner_nltk']).size().reset_index(name='counts'
                                               ).sort_values(by=['counts'], 
                                                             ascending=False)

,tweet_ner_nltk,counts
1,GPE,5601
4,PERSON,649
2,GSP,22
3,ORGANIZATION,3
0,FACILITY,2


## 3.3

lib   | recognize NER | count

spicy | 17            | почти все 

nltk  | 5             | мало

nltk - во многих источниках пишут , что не производительная и библилтека с низким результатом